## **1. Load Data**

In [4]:
import pandas as pd

df = pd.read_csv('../Data/vietnamese_news_preprocessed.csv')

In [2]:
df.head()

,date,source,title,content,category,url,date_clean,raw_word_count,clean_content,clean_title,token_count
0,16/8/2024,VnExpress,Những lỗi cần tránh khi thiết kế giếng trời,Giếng trời là không gian thông suốt từ tầng tr...,Bất động sản,https://vnexpress.net/nhung-loi-can-tranh-khi-...,2024-08-16,731,giếng trời không_gian thông_suốt tầng mái phươ...,lỗi thiết_kế giếng trời,277
1,19/12/2023,VnExpress,Căn hộ thông tầng 240 m2 hướng sông Hồng với n...,Căn hộ thông tầng (duplex) có quy mô 2 tầng vớ...,Bất động sản,https://vnexpress.net/can-ho-thong-tang-240-m2...,2023-12-19,858,căn_hộ thông tầng duplex quy_mô tầng tổng diện...,căn_hộ thông tầng m hướng sông hồng nội_thất t...,398
2,7/11/2024,VnExpress,'Không được kiểm tra hiện trạng công trình khi...,"Nội dung được ông Nguyễn Toàn Thắng, Giám đốc ...",Bất động sản,https://vnexpress.net/khong-duoc-kiem-tra-hien...,2024-11-07,481,nội_dung nguyễn_toàn_thắng giám_đốc sở tài_ngu...,kiểm_tra hiện_trạng công_trình đăng_ký biến_độ...,188
3,11/6/2024,VnExpress,Hà Nội đề xuất xây mới 9 khu nhà ở xã hội,Thông tin này được Sở Xây dựng Hà Nội nêu tron...,Bất động sản,https://vnexpress.net/ha-noi-de-xuat-xay-moi-9...,2024-06-11,538,thông_tin sở xây_dựng hà_nội nêu báo_cáo tình_...,hà_nội đề_xuất xây khu xã_hội,176
4,29/1/2025,VnExpress,Biệt thự 700 m2 đưa thiên nhiên vào trung tâm,"Căn biệt thự có quy mô hai tầng, được xây dựng...",Bất động sản,https://vnexpress.net/biet-thu-700-m2-dua-thie...,2025-01-29,732,biệt_thự quy_mô hai tầng xây_dựng khu đất rộng...,biệt thự m thiên_nhiên trung_tâm,345


In [3]:
df['clean_content'].isnull().sum()

np.int64(0)

In [4]:
df['clean_title'].isnull().sum()

np.int64(27)

## **2. Xác Định Dữ Liệu Đầu Vào**

In [5]:
X = df[['clean_content', 'clean_title', 'source', 'token_count']]
y = df['category']
X['clean_title'] = X['clean_title'].fillna('')

C:\Users\Admin\AppData\Local\Temp\ipykernel_8928\3119570904.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['clean_title'] = X['clean_title'].fillna('')


## **3. Chọn Embeddings**

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=True)
model = AutoModel.from_pretrained('vinai/phobert-base')

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

## **4. Chuyển Text Thành Embeddings**

In [2]:
import numpy as np

def get_embeddings_batch(text_list, batch_size=8, max_length=256):
    embeddings = []
    for i in range(0, len(text_list), batch_size):
        batch_texts = text_list[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=max_length
        )
        inputs = {k:v.to(device) for k,v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embs = outputs.last_hidden_state[:,0,:].cpu().numpy()
        embeddings.append(cls_embs)
    return np.vstack(embeddings)

In [13]:
embedding = get_embeddings_batch(X['clean_content'][0])
embedding[:10]

array([[-0.38192302,  0.03760935, -0.48352212, ...,  0.26601052,
        -0.0741476 , -0.4214403 ],
       [ 0.3879543 ,  0.08918248, -0.5894066 , ..., -0.24664897,
         0.1273549 ,  0.38515544],
       [-0.12231984,  0.310655  , -0.5432254 , ...,  0.29389566,
        -0.23620209, -0.3951552 ],
       ...,
       [ 0.29702258,  0.20962156, -1.0126009 , ...,  0.2714441 ,
         0.13536559,  0.06980564],
       [-0.31715542, -0.13974375, -0.35844716, ...,  0.12474455,
         0.10171278, -0.29053935],
       [-0.3784134 ,  0.20758712, -0.8902823 , ...,  0.26405233,
        -0.06176281, -0.17362088]], shape=(10, 768), dtype=float32)

## **5. Tạo Pipeline cho FE**

In [15]:
X.shape

(30110, 4)

In [16]:
title_embeddings = get_embeddings_batch(X['clean_title'].tolist(),
                                         batch_size=8, 
                                         max_length=64)

In [17]:
content_embeddings = get_embeddings_batch(X['clean_content'].tolist(),
                                           batch_size=8, 
                                           max_length=256)

In [18]:
source_feat = np.array([[hash(s) % 1000] for s in X['source']])
token_count_feat = np.array([[c] for c in X['token_count']])

In [19]:
X_features = np.hstack([
    title_embeddings,
    content_embeddings,
    source_feat,
    token_count_feat
])

## **6. Lưu Embeddings Để Tái Sử Dụng**

In [20]:
np.save('../pipelines/X_features.npy', X_features)